#Setup Code

In [7]:
import ee
import geemap.core as geemap
import numpy as np
import os


# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='examplegeostat')

imageCoordinates = [[-96.8, 28.0],
[-85.3, 28.0],
[-85.3, 22],
[-96.8, 22]]
polygonArea = ee.Geometry.Polygon(imageCoordinates);

In [2]:
# Band aliases.
BLUE = 'CMI_C01'
RED = 'CMI_C02'
VEGGIE = 'CMI_C03'
CIRRUS = 'CMI_C04'
SNOW = 'CMI_C05'
PARTICLE = 'CMI_C06'
SHORTWAVE = 'CMI_C07'
UPPER = 'CMI_C08'
MID = 'CMI_C09'
LOWER = 'CMI_C10'
CLOUDTOP = 'CMI_C11'
OZONE = 'CMI_C12'
LONGWAVE = 'CMI_C13'
SEATEMP = 'CMI_C14'
ASH = 'CMI_C15'
CO2 = 'CMI_C16'
GREEN = 'GREEN'


# 16 pairs of CMI and DQF followed by Bah 2018 synthetic green.
# Band numbers in the EE asset, 0-based.
NUM_BANDS = 33

# Skipping the interleaved DQF bands.
BLUE_BAND_INDEX = (1 - 1) * 2
RED_BAND_INDEX = (2 - 1) * 2
VEGGIE_BAND_INDEX = (3 - 1) * 2
GREEN_BAND_INDEX = NUM_BANDS - 1

# Visualization range for GOES RGB.
GOES_MIN = 0.0
GOES_MAX = 0.7  # Alternatively 1.0 or 1.3.
GAMMA = 1.3

goesClouds = {
    'bands': [CIRRUS, CO2, CLOUDTOP],
    'min': [0,92,127],
    'max': [1.3,319,342],
    'gamma': [1,0.5,2]
}

goesRgbViz = {
    'bands': [RED,VEGGIE,BLUE],
    'min': 0,
    'max': 1.3,
    'gamma': 1
}

def apply_scale_and_offset(image):
    image = ee.Image(image)
    bands = [None] * NUM_BANDS
    for i in range(1, 17):
        band_name = 'CMI_C' + str(100 + i)[-2:]
        offset = ee.Number(image.get(band_name + '_offset'))
        scale = ee.Number(image.get(band_name + '_scale'))
        bands[(i - 1) * 2] = image.select(band_name).multiply(scale).add(offset)

        dqf_name = 'DQF_C' + str(100 + i)[-2:]
        bands[(i - 1) * 2 + 1] = image.select(dqf_name)

    # Bah, Gunshor, Schmit, Generation of GOES-16 True Color Imagery without a
    # Green Band, 2018. https://doi.org/10.1029/2018EA000379
    # Green = 0.45 * Red + 0.10 * NIR + 0.45 * Blue
    green1 = bands[RED_BAND_INDEX].multiply(0.45)
    green2 = bands[VEGGIE_BAND_INDEX].multiply(0.10)
    green3 = bands[BLUE_BAND_INDEX].multiply(0.45)
    green = green1.add(green2).add(green3)
    bands[GREEN_BAND_INDEX] = green.rename(GREEN)
    return ee.Image(ee.Image(bands).copyProperties(image, image.propertyNames()))

In [25]:
goesCollection = ee.ImageCollection('NOAA/GOES/16/MCMIPF').sort('system:index')
ids = goesCollection.aggregate_array('system:index').getInfo()

In [4]:
goesCollection.propertyNames()

#Testing Code

In [5]:
print(len(ids))

326175


In [6]:
collection = 'NOAA/GOES/16/MCMIPF/'
asset_id = collection + ids[1000]
imageNoOffset = ee.Image(asset_id)
image = apply_scale_and_offset(asset_id)
imageNoOffset.propertyNames()

In [ ]:
ee.Date(image.get('system:time_start')).format('YYYY-MM-dd_HH:mm:ss').getInfo()

'2017-07-10_21:15:38'

In [ ]:
clouds_Mask_Clouds = image.select(CLOUDTOP).lt(280)

cloudsImage = image.updateMask(clouds_Mask_Clouds)

In [ ]:
map = geemap.Map(center=[25,-89],zoom=6,add_google_map=False,
                 search_ctrl=False,zoom_ctrl=False,draw_ctrl=False,scale_ctrl=False)

map.addLayer(cloudsImage)
map.addLayer(image, goesRgbViz,'rgb',False)
display(map)

Map(center=[25, -89], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [ ]:
cloudsImage_vis = cloudsImage.visualize(**{'bands':[CIRRUS,CO2,CLOUDTOP],'min':[0,92,127],
    'max': [1.3,319,342],
    'gamma': [1,0.5,2]})
map = geemap.Map(center=[25,-89],zoom=6,add_google_map=False,
                 search_ctrl=False,zoom_ctrl=False,draw_ctrl=False,scale_ctrl=False)

map.addLayer(cloudsImage_vis)
map.addLayer(image, goesRgbViz,'rgb',False)
display(map)

Map(center=[25, -89], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [ ]:
image_vis = image.visualize(**{'bands':[RED,VEGGIE,BLUE],'min':0,
    'max': 1.3,
    'gamma': 1})

In [ ]:
name = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd_HH:mm:ss').getInfo()

info = cloudsImage_vis.getInfo()
crsGOES = info['bands'][0]['crs']
crs_transform = info['bands'][0]['crs_transform']
imageCoordinates = [[-96.8, 28.0], [-85.3, 28.0], [-85.3, 22], [-96.8, 22]]
geometry = ee.Geometry.Rectangle(-96.8,22,-85.3,28)
nativeProjection = cloudsImage_vis.projection()

task = ee.batch.Export.image.toDrive(
    image=cloudsImage_vis,
    description='Masked ' + name,
    crs=crsGOES,
    crsTransform=crs_transform,
    region=geometry,
    folder = 'Cloud Data'
)
task.start()

info = image_vis.getInfo()
crsGOES = info['bands'][0]['crs']
crs_transform = info['bands'][0]['crs_transform']
imageCoordinates = [[-96.8, 28.0], [-85.3, 28.0], [-85.3, 22], [-96.8, 22]]
geometry = ee.Geometry.Rectangle(-96.8,22,-85.3,28)
nativeProjection = image_vis.projection()

task = ee.batch.Export.image.toDrive(
    image=image_vis,
    description='RGB ' + name,
    crs=crsGOES,
    crsTransform=crs_transform,
    region=geometry,
    folder = 'Cloud Data'
)
task.start()

In [36]:
def unique(list1):
  # initialize a null list
  unique_list = []

  # traverse for all elements
  for x in list1:
    # check if exists in unique_list or not
    if x not in unique_list:
      unique_list.append(x)
  return unique_list

In [37]:
names = []
for filename in os.listdir('/content/drive/MyDrive/ENEE439D Group Project/Cloud Data /PNGs'):
  if filename.startswith('Masked'):
    names.append(filename)
names.sort(reverse=True)
latest_id = len(names)
latest = names[0]
print(latest)

Masked 2017-07-29_02:00:37.png


In [ ]:
unique_list = unique(names)
print(len(unique_list))

In [32]:
print(latest_id)

2001


In [26]:
asset_id = collection + ids[]
img = ee.Image(asset_id)

In [28]:
ee.Date(img.get('system:time_start')).format('YYYY-MM-dd_HH:mm:ss').getInfo()

'2017-07-31_20:45:37'

In [13]:
print(ee.Date(image.get('system:time_start')).format('YYYY-MM-dd_HH:mm:ss').getInfo())

2017-07-21_08:15:38


#Code to Run on Server

In [ ]:
import ee
import geemap.core as geemap
import numpy as np


# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='examplegeostat')

# Band aliases.
BLUE = 'CMI_C01'
RED = 'CMI_C02'
VEGGIE = 'CMI_C03'
CIRRUS = 'CMI_C04'
SNOW = 'CMI_C05'
PARTICLE = 'CMI_C06'
SHORTWAVE = 'CMI_C07'
UPPER = 'CMI_C08'
MID = 'CMI_C09'
LOWER = 'CMI_C10'
CLOUDTOP = 'CMI_C11'
OZONE = 'CMI_C12'
LONGWAVE = 'CMI_C13'
SEATEMP = 'CMI_C14'
ASH = 'CMI_C15'
CO2 = 'CMI_C16'
GREEN = 'GREEN'


# 16 pairs of CMI and DQF followed by Bah 2018 synthetic green.
# Band numbers in the EE asset, 0-based.
NUM_BANDS = 33

# Skipping the interleaved DQF bands.
BLUE_BAND_INDEX = (1 - 1) * 2
RED_BAND_INDEX = (2 - 1) * 2
VEGGIE_BAND_INDEX = (3 - 1) * 2
GREEN_BAND_INDEX = NUM_BANDS - 1

# Visualization range for GOES RGB.
GOES_MIN = 0.0
GOES_MAX = 0.7  # Alternatively 1.0 or 1.3.
GAMMA = 1.3

goesClouds = {
    'bands': [CIRRUS, CO2, CLOUDTOP],
    'min': [0,92,127],
    'max': [1.3,319,342],
    'gamma': [1,0.5,2]
}

goesRgbViz = {
    'bands': [RED,VEGGIE,BLUE],
    'min': 0,
    'max': 1.3,
    'gamma': 1
}

def apply_scale_and_offset(image):
    image = ee.Image(image)
    bands = [None] * NUM_BANDS
    for i in range(1, 17):
        band_name = 'CMI_C' + str(100 + i)[-2:]
        offset = ee.Number(image.get(band_name + '_offset'))
        scale = ee.Number(image.get(band_name + '_scale'))
        bands[(i - 1) * 2] = image.select(band_name).multiply(scale).add(offset)

        dqf_name = 'DQF_C' + str(100 + i)[-2:]
        bands[(i - 1) * 2 + 1] = image.select(dqf_name)

    # Bah, Gunshor, Schmit, Generation of GOES-16 True Color Imagery without a
    # Green Band, 2018. https://doi.org/10.1029/2018EA000379
    # Green = 0.45 * Red + 0.10 * NIR + 0.45 * Blue
    green1 = bands[RED_BAND_INDEX].multiply(0.45)
    green2 = bands[VEGGIE_BAND_INDEX].multiply(0.10)
    green3 = bands[BLUE_BAND_INDEX].multiply(0.45)
    green = green1.add(green2).add(green3)
    bands[GREEN_BAND_INDEX] = green.rename(GREEN)
    return ee.Image(ee.Image(bands).copyProperties(image, image.propertyNames()))

collection = 'NOAA/GOES/16/MCMIPF/'
goesCollection = ee.ImageCollection('NOAA/GOES/16/MCMIPF')
ids = goesCollection.aggregate_array('system:index').getInfo()
id_batch = ids[latest_id:]

for id in id_batch:
  asset_id = collection + id
  image = apply_scale_and_offset(asset_id)

  clouds_Mask_Clouds = image.select(CLOUDTOP).lt(280)
  cloudsImage = image.updateMask(clouds_Mask_Clouds)

  cloudsImage_vis = cloudsImage.visualize(**{'bands':[CIRRUS,CO2,CLOUDTOP],'min':[0,92,127],
    'max': [1.3,319,342],
    'gamma': [1,0.5,2]})
  image_vis = image.visualize(**{'bands':[RED,VEGGIE,BLUE],'min':0,
    'max': 1.3,
    'gamma': 1})

  name = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd_HH:mm:ss').getInfo()

  info = cloudsImage_vis.getInfo()
  crsGOES = info['bands'][0]['crs']
  crs_transform = info['bands'][0]['crs_transform']
  geometry = ee.Geometry.Rectangle(-96.8,22,-85.3,28)
  nativeProjection = cloudsImage_vis.projection()

  task = ee.batch.Export.image.toDrive(
      image=cloudsImage_vis,
      description='Masked ' + name,
      crs=crsGOES,
      crsTransform=crs_transform,
      region=geometry
  )
  task.start()

  info = image_vis.getInfo()
  crsGOES = info['bands'][0]['crs']
  crs_transform = info['bands'][0]['crs_transform']
  geometry = ee.Geometry.Rectangle(-96.8,22,-85.3,28)
  nativeProjection = image_vis.projection()

  task = ee.batch.Export.image.toDrive(
      image=image_vis,
      description='RGB ' + name,
      crs=crsGOES,
      crsTransform=crs_transform,
      region=geometry
  )
  task.start()


EEException: Too many tasks already in the queue (3000). Please wait for some of them to complete.

# Test Server Code

In [ ]:
import ee
import geemap.core as geemap
import numpy as np


# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='examplegeostat')

# Band aliases.
BLUE = 'CMI_C01'
RED = 'CMI_C02'
VEGGIE = 'CMI_C03'
CIRRUS = 'CMI_C04'
SNOW = 'CMI_C05'
PARTICLE = 'CMI_C06'
SHORTWAVE = 'CMI_C07'
UPPER = 'CMI_C08'
MID = 'CMI_C09'
LOWER = 'CMI_C10'
CLOUDTOP = 'CMI_C11'
OZONE = 'CMI_C12'
LONGWAVE = 'CMI_C13'
SEATEMP = 'CMI_C14'
ASH = 'CMI_C15'
CO2 = 'CMI_C16'
GREEN = 'GREEN'


# 16 pairs of CMI and DQF followed by Bah 2018 synthetic green.
# Band numbers in the EE asset, 0-based.
NUM_BANDS = 33

# Skipping the interleaved DQF bands.
BLUE_BAND_INDEX = (1 - 1) * 2
RED_BAND_INDEX = (2 - 1) * 2
VEGGIE_BAND_INDEX = (3 - 1) * 2
GREEN_BAND_INDEX = NUM_BANDS - 1

# Visualization range for GOES RGB.
GOES_MIN = 0.0
GOES_MAX = 0.7  # Alternatively 1.0 or 1.3.
GAMMA = 1.3

goesClouds = {
    'bands': [CIRRUS, CO2, CLOUDTOP],
    'min': [0,92,127],
    'max': [1.3,319,342],
    'gamma': [1,0.5,2]
}

goesRgbViz = {
    'bands': [RED,VEGGIE,BLUE],
    'min': 0,
    'max': 1.3,
    'gamma': 1
}

def apply_scale_and_offset(image):
    image = ee.Image(image)
    bands = [None] * NUM_BANDS
    for i in range(1, 17):
        band_name = 'CMI_C' + str(100 + i)[-2:]
        offset = ee.Number(image.get(band_name + '_offset'))
        scale = ee.Number(image.get(band_name + '_scale'))
        bands[(i - 1) * 2] = image.select(band_name).multiply(scale).add(offset)

        dqf_name = 'DQF_C' + str(100 + i)[-2:]
        bands[(i - 1) * 2 + 1] = image.select(dqf_name)

    # Bah, Gunshor, Schmit, Generation of GOES-16 True Color Imagery without a
    # Green Band, 2018. https://doi.org/10.1029/2018EA000379
    # Green = 0.45 * Red + 0.10 * NIR + 0.45 * Blue
    green1 = bands[RED_BAND_INDEX].multiply(0.45)
    green2 = bands[VEGGIE_BAND_INDEX].multiply(0.10)
    green3 = bands[BLUE_BAND_INDEX].multiply(0.45)
    green = green1.add(green2).add(green3)
    bands[GREEN_BAND_INDEX] = green.rename(GREEN)
    return ee.Image(ee.Image(bands).copyProperties(image, image.propertyNames()))

collection = 'NOAA/GOES/16/MCMIPF/'
goesCollection = ee.ImageCollection('NOAA/GOES/16/MCMIPF')
ids = goesCollection.aggregate_array('system:index').getInfo()
ids_small = ids[1:51]

for id in ids_small:
  asset_id = collection + id
  image = apply_scale_and_offset(asset_id)

  clouds_Mask_Clouds = image.select(CLOUDTOP).lt(280)
  cloudsImage = image.updateMask(clouds_Mask_Clouds)

  cloudsImage_vis = cloudsImage.visualize(**{'bands':[CIRRUS,CO2,CLOUDTOP],'min':[0,92,127],
    'max': [1.3,319,342],
    'gamma': [1,0.5,2]})
  image_vis = image.visualize(**{'bands':[RED,VEGGIE,BLUE],'min':0,
    'max': 1.3,
    'gamma': 1})

  name = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd_HH:mm:ss').getInfo()

  info = cloudsImage_vis.getInfo()
  crsGOES = info['bands'][0]['crs']
  crs_transform = info['bands'][0]['crs_transform']
  geometry = ee.Geometry.Rectangle(-96.8,22,-85.3,28)
  nativeProjection = cloudsImage_vis.projection()

  task = ee.batch.Export.image.toDrive(
      image=cloudsImage_vis,
      description='Masked ' + name,
      crs=crsGOES,
      crsTransform=crs_transform,
      region=geometry
  )
  task.start()

  info = image_vis.getInfo()
  crsGOES = info['bands'][0]['crs']
  crs_transform = info['bands'][0]['crs_transform']
  geometry = ee.Geometry.Rectangle(-96.8,22,-85.3,28)
  nativeProjection = image_vis.projection()

  task = ee.batch.Export.image.toDrive(
      image=image_vis,
      description='RGB ' + name,
      crs=crsGOES,
      crsTransform=crs_transform,
      region=geometry
  )
  task.start()